In [ ]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})

<h1>Linear Programming: Graph drawing with springs</h1>

Last Time: We saw introductory examples of linear programming, based on the max flow problem.

This Time: We introduce another example based on Tutte's algorithm for drawing graphs.

<h1>Drawing Graphs</h1>

Let $G$ be a graph. By definition, $G$ is just a set of vertices with an adjacency relation. The vertices do not have spatial locations. By "drawing a graph" we mean assigning spatial locations in $\mathbb{R}^2$ to $V(G)$ and curves to each edge of $G$ that begin and end at the respective vertices. A graph drawing is planar if its edges do not cross, and a graph is planar if it has a planar drawing.

The Tutte algorithm for drawing planar graphs without crossings can be written in terms of matrices. See [here](https://www.youtube.com/watch?v=mEzPPMhR8XE) for a brief video explanation. See also Graphs and Geometry by Laszlo Lovasz.

The idea is to:

1. Identify a minimal cycle to serve as the outer face.
    - Use a graph search algorithm.
2. Fix the positions of the vertices on the outer face.
3. Design a system of linear equations that models the edges as rubber bands with spring constants $c_{u,v}$. (We can assume this mass is $c_{u,v}=1$ for each edge.)
4. Find the unique solution to the system of linear equations.

We now elaborate on steps $3$ and $4$.

<h3>Designing a system of equations</h3>

Suppose that $v\in V(G)$ is a vertex and we would like to position it at $p_v=(x_v,y_v)$, where $(x_v,y_v)\in \mathbb{R}^2$ are the coordinates of $v$ in the drawing.

The amount of potential energy stored in the rubber band representing an edge $v\sim u$ is $c_{v,u}|p_v - p_u|^2$, so the total potential energy is $\sum_{u,v\in E(G)} c_{v,u}|p_v - p_u|^2$. This is a convex equation and so has a unique minimum. The minimum is achieved when each vertex is in balance, so that the forces of its neighbors cancel out. Therefore, we introduce the constraints that for each vertex $v\in V(G)$, $\sum_{u\in \mathcal{N}(v)} c_{u,v} (p_u-p_v)=0$. This is really two constraints- one for each coordinate.


\begin{align*}
    0 = \sum_{u \in \mathcal{N}(v)} c_{u,v} (x_u -x_v) =\sum_{u \in \mathcal{N}(v)} c_{u,v} (y_u -y_v)
\end{align*}

Let $k$ be the number of vertices that are pinned in step $2$. Then there are $2(n-k)$ variables with $2(n-k)$ linear constraints. We can put the constraints into the rows of a matrix $A\in \mathbb{R}^{2(n-k) \times 2(n-k)}$. We denote the entries of $A$ by $A_{v,u,z}$ where $v,u\in V(G)$ and $z\in \{x,y\}$. The two rows corresponding to the constraints on an unpinned vertex $v$ are given by $A_{v,u,z}=\begin{cases}-\sum_{u\in \mathcal{N}(v)} c_{u,v} &\text{if }v=u\\ c_{u,v} &\text{if } u\in \mathcal{N}(v)\\ 0 &\text{otherwise} \end{cases}$.

The constraint vector is $b=\bold{0}$. Our constraints are the linear constraints $Ap = \bold{0}$, where $p = (p_v)_{v\in V(G)}$ is a vector that encodes the positions of all of the unpinned vertices. If no vertices are pinned ($k=0$), then each row of $A$ sums to $0$, which demonstrates that the constraints are not all independent, and the solution is not unique. 

Theorem (Tutte): if $G$ is a $3$-connected planar graph and we pin a face of the graph, then the Tutte embedding is unique when we set $c_{u,v}=1$ for all $u,v\in V(G)$.

<h3>Linear programming and the Tutte Drawing</h3>

The Tutte drawing is a simple example of linear programming where the feasible set contains only a single element. The more general technique of linear programming suggests that we can enlarge the feasible set by allowing more flexibility for the pinned vertices. Then we can optimize over the pinned locations.

<h3>Graph Drawing and connectivity</h3>

In order to represent a graph visually, we must understand the graph well enough so that it can be displayed clearly. In this sense, every algorithm for drawing graphs can be turned into an algorithm to calculate properties of the graphs. In particular, the Tutte drawing algorithm can be modified into an algorithm to calculate vertex connectivity.

Recall that the vertex connectivity of a graph is the minimum number of vetices that can be removed to disconnect the graph. When the graph is $3$-connected, Tutte's theorem says that it can be drawn in $\mathbb{R}^2$ in general position. Laszlo Lovasz proved a stronger statement: If the graph is $k$ connected, and we choose the edge coefficients $c_{u,v}$ and pinned vertices at random, then with high probability the Tutte drawing in $\mathbb{R}^{k-1}$ can be drawn in general position. But if the graph is not $k$ connected, then with high probability, the resulting graph will not be in general position.

This provides the basis for a fast algorithm to decide whether a graph is $k$-connected.

To illustrate the idea, let $G$ be a graph that is only $2$-connected. To check that it is not $3$ connected, we draw it in the plane and check if the resulting vertices are in general position.

<img src="figures/2_connected.png" width="30%">

The green vertices are pinned. If the rubber bands are allowed to contract, the region in the middle of the graph will not be in general position.

After a long, detailed analysis, the work of the algorithm is $\tilde{O}(M(V) + V(M(k)))$, where $M(n)=n^{2.37}$ is the cost of multiplying matrices, $\tilde{O}(\cdot)$ is like the big-Oh, but might hide logarithmic factors, and $k$ is the amount of connectivity that we want to test.